<a href="https://colab.research.google.com/github/Manudrel/Reconhecimento-Facial/blob/main/Reconhecimento_facial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EXPLICAÇÃO**
Foi utilizado como base para habilitar a webcam no Google Colab o NoteBook disponiblizado pelo The AI Guy no vídeo: How to Use Webcam In Google Colab for Images and Video (FACE DETECTION)

[Link do Colab do The AI Guy](https://colab.research.google.com/drive/1QnC7lV7oVFk5OZCm75fqbLAfD9qBy9bw?usp=sharing)
---

# **Import das Dependencias**

In [ ]:
!pip install -q ultralytics roboflow

In [ ]:
from IPython.display import display, HTML, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import logging
import os
import sys
from contextlib import contextmanager, redirect_stdout, redirect_stderr
from ultralytics import YOLO
from roboflow import Roboflow

# **Parte da Câmera — Funções básicas**
Esta seção é responsável por habilitar a webcam no Google Colab e preparar o ambiente para capturar imagens em tempo real.
---

In [ ]:
video_html = """
<video id="video" width=640 height=480 autoplay muted></video>
<canvas id="canvas" width=640 height=480 style="position:absolute;top:0;left:0;"></canvas>
<script>
var video = document.getElementById('video');
navigator.mediaDevices.getUserMedia({ video: true })
  .then(function(stream) {
    video.srcObject = stream;
    video.play();
  });
</script>
"""

display(HTML(video_html))

In [ ]:
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def bbox_to_bytes(bbox_array):
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    bbox_PIL.save(iobuf, format='png')
    bbox_bytes = 'data:image/png;base64,{}'.format(
        str(b64encode(iobuf.getvalue()), 'utf-8'))
    return bbox_bytes

def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data


In [ ]:
def video_stream():
    display(Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var imgElement;
        var labelElement;
        var pendingResolve = null;
        var shutdown = false;

        function removeDom() {
            stream.getVideoTracks()[0].stop();
            video.remove();
            div.remove();
            video = null;
            div = null;
            stream = null;
            imgElement = null;
            captureCanvas = null;
            labelElement = null;
        }

        function onAnimationFrame() {
            if (!shutdown) {
                window.requestAnimationFrame(onAnimationFrame);
            }
            if (pendingResolve) {
                var result = "";
                if (!shutdown) {
                    captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
                    result = captureCanvas.toDataURL('image/jpeg', 0.8)
                }
                var lp = pendingResolve;
                pendingResolve = null;
                lp(result);
            }
        }

        async function createDom() {
            if (div !== null) return stream;

            div = document.createElement('div');
            div.style.border = '2px solid black';
            div.style.padding = '3px';
            div.style.width = '100%';
            div.style.maxWidth = '600px';
            document.body.appendChild(div);

            const modelOut = document.createElement('div');
            modelOut.innerHTML = "<span>Status:</span>";
            labelElement = document.createElement('span');
            labelElement.innerText = 'No data';
            labelElement.style.fontWeight = 'bold';
            modelOut.appendChild(labelElement);
            div.appendChild(modelOut);

            video = document.createElement('video');
            video.style.display = 'block';
            video.width = div.clientWidth - 6;
            video.setAttribute('playsinline', '');
            video.onclick = () => { shutdown = true; };
            stream = await navigator.mediaDevices.getUserMedia({video: { facingMode: "user"}});
            div.appendChild(video);

            imgElement = document.createElement('img');
            imgElement.style.position = 'absolute';
            imgElement.style.zIndex = 1;
            imgElement.onclick = () => { shutdown = true; };
            div.appendChild(imgElement);

            const instruction = document.createElement('div');
            instruction.innerHTML = '<span style="color: red; font-weight: bold;">Clique para parar</span>';
            div.appendChild(instruction);
            instruction.onclick = () => { shutdown = true; };

            video.srcObject = stream;
            await video.play();

            captureCanvas = document.createElement('canvas');
            captureCanvas.width = 640;
            captureCanvas.height = 480;
            window.requestAnimationFrame(onAnimationFrame);

            return stream;
        }

        async function stream_frame(label, imgData) {
            if (shutdown) {
                removeDom();
                shutdown = false;
                return '';
            }

            stream = await createDom();

            if (label != "") {
                labelElement.innerHTML = label;
            }

            if (imgData != "") {
                var videoRect = video.getClientRects()[0];
                imgElement.style.top = videoRect.top + "px";
                imgElement.style.left = videoRect.left + "px";
                imgElement.style.width = videoRect.width + "px";
                imgElement.style.height = videoRect.height + "px";
                imgElement.src = imgData;
            }

            var result = await new Promise(function(resolve, reject) {
                pendingResolve = resolve;
            });
            shutdown = false;

            return {
                'img': result
            };
        }
    '''))

# **Criação/Treinamento do Modelo — Detecção de rostos com YOLO**
Nesta parte, o modelo YOLO é criado e treinado com um dataset do roboflow

In [ ]:
rf = Roboflow(api_key="COLOQUE_SUA_API_KEY") # API KEY (Chave pessoal do roboflow)
project = rf.workspace("COLOQUE_SEU_WORKSPACE").project("mussergesichterzensieren-rlkcm") # rf.workscape(nome_do_usuário).project(nome_do_projeto)
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
model = YOLO("yolov8n.pt")

model.train(
     data = "/content/MusserGesichterZensieren-1/data.yaml",
     epochs = 20,
     imgsz = 640,
 )

# **Loop Principal — Detecção de rostos com YOLO**
Nesta parte, o modelo YOLO é integrado ao fluxo da câmera para fazer a detecção de rostos em tempo real.

In [ ]:
@contextmanager
def suppress_all_output():
    with open(os.devnull, 'w') as devnull:
        with redirect_stdout(devnull), redirect_stderr(devnull):
            logging.disable(logging.CRITICAL)
            try:
                yield
            finally:
                logging.disable(logging.NOTSET)

In [ ]:
video_stream()


label_html = 'Detectando rostos com YOLO...'
bbox = ''

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"])
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    with suppress_all_output():
        results = model(img)

    # Desenha bounding boxes
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            bbox_array = cv2.rectangle(bbox_array, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Cria overlay com transparência
    bbox_array[:, :, 3] = (bbox_array.max(axis=2) > 0).astype(np.uint8) * 255
    bbox = bbox_to_bytes(bbox_array)

# **Teste único do modelo treinado (YOLOv8)**

Esta célula realiza um **teste simples com imagem estática** para verificar se o modelo YOLO treinado está funcionando corretamente.=
---

In [ ]:
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
def mostrar(frame):
    imagem = cv2.imread(frame)
    if imagem is None:
        print(f"Erro: imagem '{frame}' não encontrada.")
        return
    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
uploaded = files.upload()
image_path = next(iter(uploaded))

In [ ]:
results = model(image_path)
results[0].save(filename='predictions.jpg')

In [ ]:
mostrar('predictions.jpg')